In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis= 1, inplace= True)

In [4]:
###encode the gender using LabelEncoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
## encode the geography using One Hot Encoder
onehot_encoder_geography = OneHotEncoder(sparse_output=False)
geography_encoded = onehot_encoder_geography.fit_transform(data[['Geography']])

In [6]:
df1 = pd.DataFrame(geography_encoded, columns=onehot_encoder_geography.get_feature_names_out())

In [7]:
df = pd.concat([data.drop('Geography', axis =1), df1], axis =1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
##split the indep features and target
X = df.drop('EstimatedSalary', axis=1)
y = df['EstimatedSalary']

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((8000, 12), (2000, 12))

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
import pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geography, file)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [14]:
###ANN regression problem
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [15]:
model = Sequential([Dense(64, activation='relu', input_shape = (X_train.shape[1],)), Dense(32, activation='relu'), Dense(1)])

c:\Users\Omkar\Downloads\implementations\ANN_classification\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
##compile the model
model.compile(optimizer='adam', loss = 'mean_absolute_error', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir = 'regressionlogs/fit/'+datetime.datetime.now().strftime('%Y%m%d-%H%M%D')
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [18]:
###set up Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

In [19]:
###train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 99724.5938 - mae: 99724.5938 - val_loss: 98476.3438 - val_mae: 98476.3438
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 101019.2734 - mae: 101019.2734 - val_loss: 96669.6016 - val_mae: 96669.6016
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97835.8750 - mae: 97835.8750 - val_loss: 92078.1953 - val_mae: 92078.1953
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 92132.8047 - mae: 92132.8047 - val_loss: 84597.8984 - val_mae: 84597.8984
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 84523.2031 - mae: 84523.2031 - val_loss: 75216.0000 - val_mae: 75216.0000
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 75489.7266 - mae: 75489.7266 - val_loss: 66054.5625 - val_mae: 66054.5625
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64030.1914 - mae: 64030.1914 - val_loss: 58634.3555 - val_mae: 58634.3555
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [20]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit/20250112-200401/12/25

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [28]:
##Evaluate model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE :{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51088.1211 - mae: 51088.1211
Test MAE :50196.7421875


In [29]:
model.save('regression_model.h5')